In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("events") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0") \
    .getOrCreate()

In [2]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092,kafka2:19093,kafka3:19094") \
  .option("subscribe", "events") \
  .load()

In [23]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
dsraw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092,kafka2:19093,kafka3:19094") \
  .option("subscribe", "events") \
  .option("startingOffsets", "earliest") \
  .load()

In [7]:
ds = dsraw.selectExpr("CAST(value AS STRING)")

In [8]:
rawQuery = dsraw \
        .writeStream \
        .queryName("qraw")\
        .format("memory")\
        .start()


In [9]:
raw = spark.sql("select * from qraw")
raw.show()

+----+--------------------+------+---------+------+--------------------+-------------+
| key|               value| topic|partition|offset|           timestamp|timestampType|
+----+--------------------+------+---------+------+--------------------+-------------+
|null|[7B 22 74 69 6D 6...|events|        1|     0|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     1|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     2|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     3|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     4|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     5|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     6|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1|     7|2023-11-18 15:47:...|            0|
|null|[7B 22 74 69 6D 6...|events|        1

In [10]:
eventsQuery = ds \
        .writeStream \
        .queryName("events")\
        .format("memory")\
        .start()

In [11]:
alerts = spark.sql("select * from events")
alerts.show()

+--------------------+
|               value|
+--------------------+
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
|{"timestamp": 170...|
+--------------------+
only showing top 20 rows

